In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class SVC:
    
    def __init__(self, eta=0.01, reg='l2', reg_coeff=0.001, n_iter=1000):
        self.eta = eta
        self.reg = reg
        self.reg_coeff = reg_coeff
        self.n_iter = n_iter
        self.w = None
        self.b = None
        self.loss = None
        
    def fit(self, X, y):
        N,D = X.shape
        y = np.where(y <= 0, -1, 1)
        
        self.w = np.zeros(D)
        self.b = 0
        self.loss = []
        
        for j in range(self.n_iter):
            error = 0
            for i in range(N):
                condition = y[i] * (np.dot(X[i],self.w) + self.b)
                
                if condition>=1:
                    if self.reg == 'l1':
                        dw = np.sign(self.w)
                    else: 
                        dw = self.w
                    db = 0
                    
                else:
                    if self.reg == 'l1':
                        dw = self.reg_coeff * np.sign(self.w) - np.dot(X[i],y[i])
                    else : 
                        dw = self.reg_coeff * self.w - np.dot(X[i],y[i]) 
                    db = -y[i]
                    
                self.w -= self.eta *  dw
                self.b -= self.eta * db
                error += self.hinge(condition)
                
            self.loss.append(error/N)
                
                
                
    def hinge(self,condition):
        if condition >=1:
            return 0
        else:
            return 1-condition
        
    def predict(self, X):
        y_pred = np.sign(np.dot(X, self.w) + self.b)
        p=[1 if n >= 0 else 0 for n in y_pred]
        return y_pred
    
    def score(self,X,y):
        y_pred = np.sign(np.dot(X, self.w) + self.b)
        y_pred=[1 if n >= 0 else 0 for n in y_pred]
        accuracy = np.sum(y == y_pred) / len(y)
        return accuracy
        
    def weights(self):
        return self.w,self.b
    
    def error(self):
        return np.array(self.loss)

In [3]:
df=pd.read_csv(r'heart.csv')

In [4]:
df['cp']=df['cp'].astype('object')
df['slope']=df['slope'].astype('object')
df['thal']=df['thal'].astype('object')

In [5]:
X=df.iloc[:,df.columns !='target']
y=df.iloc[:,df.columns =='target']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y, test_size=0.25 , random_state=100)

In [7]:
from feature_engine.encoding import OneHotEncoder
encoder=OneHotEncoder(variables=['cp','slope','thal'] ,drop_last=True)
X_train=encoder.fit_transform(X_train)
X_test=encoder.transform(X_test)

In [8]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [9]:
y_train=y_train.values
y_test=y_test.values

In [10]:
y_train=y_train.reshape(-1)
y_test=y_test.reshape(-1)

In [ ]:
clf = SVC()
clf.fit(X_train, y_train)
clf.score(X_test,y_test)

In [ ]:
error = clf.error()
print(error)

In [ ]:
plt.plot(error)